
## In this Notebook , we used to fill the Null values automatically and also trim the spaces around the values 

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

In [0]:
# run the setup notebook

In [0]:
%run /Workspace/Generic/Nb_SetUpMounts_JDBC

[FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales-1.csv', name='2024_04_18_4_WheelSales-1.csv', size=10992761, modificationTime=1719993217000),
 FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales.csv', name='2024_04_18_4_WheelSales.csv', size=10992762, modificationTime=1719989742000),
 FileInfo(path='dbfs:/FileStore/4_WheelSales.csv', name='4_WheelSales.csv', size=10992762, modificationTime=1719989774000),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0, modificationTime=1716203553000),
 FileInfo(path='dbfs:/FileStore/source1.csv', name='source1.csv', size=1138, modificationTime=1709555634000),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=1708600418000)]

['PipelineName', 'Pk_PipelineID', 'Pk_ActivityID', 'PipelineSourceTypes', 'ParameterValueSink', 'parameterValues2', 'ColumnSourceTypes', 'SourceColumnName', 'TargetColumnName', 'sourcecolumntypes', 'IsNullable', 'IsActive']


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive,test
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1,2-WheelSales
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null,4-WheelSales


2-WheelSales
/mnt/genericSilver/2-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericSilver/4-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericrawTest/2-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericrawTest/4-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericStage/2-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericStage/4-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2024/07/04


In [0]:
## define the function 

In [0]:
def fn_handlenull_spaces(filepaths, reference_df):

    try:

        # file_info = dbutils.fs.ls(filepaths)

        # file_paths = [file.path for file in file_info]
        # print(file_paths)

        # paths = [f"'{file.path}'" for file in file_info]

        for x in reference_df.collect():

            filenames = x['parameterValues2']
            print(filenames)
            folder_names = x['test']
            print(folder_names)

            dt_string = datetime.now().strftime("%Y/%m/%d")
            full_path = f"{filepaths}{folder_names}/{dt_string}/{filenames}"
            print(full_path)


            try:
                dbutils.fs.ls(full_path)
                print(f'Exists :: {full_path}')

                dfs = spark.read.format('parquet').option('inferSchema', True).load(full_path)

                nulls_count = dfs.select([ count( when( col(c).isNull() | isnan(c),c) ).alias(c) for c in dfs.columns  ]).first().asDict()

                total_nulls = 0

                for null_count in nulls_count.values():
                    total_nulls += null_count
                print(f"Total null values in {filenames}: {total_nulls}")

                dfs_str = dfs.fillna('unkown', subset=dfs.schema.names)

                dfs_cleaned = dfs_str.fillna(0, subset=dfs.schema.names)

                nulls_count2 = dfs_cleaned.select([count( when (col(c).isNull() | isnan(c),c  )).alias(c) for c in dfs_cleaned.columns  ])

                output_good_path = f"/mnt/genericSilver/{folder_names}/{dt_string}"

                dfs2 = dfs_cleaned.select(*[trim(col(c)).alias(c) for c in dfs_cleaned.columns])
        
                dfs2.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
                print("Done")

            except Exception as e:

                print(f'Does not exists :: {full_path}')

    except Exception as e:
        raise e



In [0]:
fn_handlenull_spaces(filepaths= '/mnt/genericraw/', reference_df= res)

2-WheelSData.parquet
2-WheelSales
/mnt/genericraw/2-WheelSales/2024/07/04/2-WheelSData.parquet
Exists :: /mnt/genericraw/2-WheelSales/2024/07/04/2-WheelSData.parquet
Total null values in 2-WheelSData.parquet: 964
Done
4-WheelSData.parquet
4-WheelSales
/mnt/genericraw/4-WheelSales/2024/07/04/4-WheelSData.parquet
Exists :: /mnt/genericraw/4-WheelSales/2024/07/04/4-WheelSData.parquet
Total null values in 4-WheelSData.parquet: 3960
Done
